# YOLOv12 Implementation for Military Aircraft Detection Dataset

This notebook implements YOLOv12 for the Military Aircraft Detection Dataset available on Kaggle. The implementation includes data preprocessing, model architecture, training, and evaluation.

## 1. Setup and Dependencies

In [ ]:
# Install required packages
!pip install -q ultralytics pandas matplotlib seaborn scikit-learn opencv-python-headless

In [ ]:
# Import necessary libraries
import os
import glob
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Dataset Exploration

In [ ]:
# Define dataset paths
DATASET_PATH = '../input/militaryaircraftdetectiondataset/dataset'
CROP_PATH = '../input/militaryaircraftdetectiondataset/crop'
ANNOTATED_SAMPLES_PATH = '../input/militaryaircraftdetectiondataset/annotated_samples'

# Check if paths exist
print(f"Dataset path exists: {os.path.exists(DATASET_PATH)}")
print(f"Crop path exists: {os.path.exists(CROP_PATH)}")
print(f"Annotated samples path exists: {os.path.exists(ANNOTATED_SAMPLES_PATH)}")

In [ ]:
# List image and annotation files
image_files = glob.glob(os.path.join(DATASET_PATH, '*.jpg'))
annotation_files = glob.glob(os.path.join(DATASET_PATH, '*.csv'))

print(f"Number of images: {len(image_files)}")
print(f"Number of annotation files: {len(annotation_files)}")

# Check class distribution
all_classes = []
for csv_file in tqdm(annotation_files[:100]):  # Sample first 100 for quick analysis
    df = pd.read_csv(csv_file)
    all_classes.extend(df['class'].tolist())

class_counts = pd.Series(all_classes).value_counts()
print(f"Number of unique classes in sample: {len(class_counts)}")

# Plot class distribution
plt.figure(figsize=(15, 8))
sns.barplot(x=class_counts.index[:20], y=class_counts.values[:20])
plt.title('Top 20 Aircraft Classes Distribution')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize some sample images with annotations
def visualize_sample(image_path, annotation_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    df = pd.read_csv(annotation_path)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    
    for _, row in df.iterrows():
        xmin, ymin, xmax, ymax = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        class_name = row['class']
        
        # Draw bounding box
        plt.gca().add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, 
                                         fill=False, edgecolor='red', linewidth=2))
        plt.text(xmin, ymin-10, class_name, color='red', fontsize=12, 
                 bbox=dict(facecolor='white', alpha=0.7))
    
    plt.title(f"Image with annotations: {os.path.basename(image_path)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize a few samples
for i in range(min(3, len(image_files))):
    img_path = image_files[i]
    ann_path = os.path.join(DATASET_PATH, os.path.basename(img_path).replace('.jpg', '.csv'))
    if os.path.exists(ann_path):
        visualize_sample(img_path, ann_path)

## 3. Data Preprocessing for YOLOv12

In [ ]:
# Create class mapping
def create_class_mapping():
    # Get all unique classes from annotation files
    all_classes = set()
    for csv_file in tqdm(annotation_files):
        try:
            df = pd.read_csv(csv_file)
            if 'class' in df.columns:
                all_classes.update(df['class'].unique())
        except Exception as e:
            print(f"Error reading {csv_file}: {e}")
    
    # Create class mapping
    class_list = sorted(list(all_classes))
    class_to_idx = {cls: idx for idx, cls in enumerate(class_list)}
    
    # Save class mapping
    with open('classes.txt', 'w') as f:
        for cls in class_list:
            f.write(f"{cls}\n")
    
    return class_list, class_to_idx

# Convert annotations to YOLO format
def convert_to_yolo_format(class_to_idx):
    os.makedirs('yolo_dataset/images/train', exist_ok=True)
    os.makedirs('yolo_dataset/images/val', exist_ok=True)
    os.makedirs('yolo_dataset/labels/train', exist_ok=True)
    os.makedirs('yolo_dataset/labels/val', exist_ok=True)
    
    # Split dataset into train and validation
    all_files = [os.path.basename(f).replace('.jpg', '') for f in image_files]
    train_files, val_files = train_test_split(all_files, test_size=0.2, random_state=42)
    
    print(f"Training samples: {len(train_files)}")
    print(f"Validation samples: {len(val_files)}")
    
    # Process training files
    for file_id in tqdm(train_files):
        img_path = os.path.join(DATASET_PATH, f"{file_id}.jpg")
        csv_path = os.path.join(DATASET_PATH, f"{file_id}.csv")
        
        if not os.path.exists(img_path) or not os.path.exists(csv_path):
            continue
        
        # Copy image to train folder
        os.system(f"cp '{img_path}' 'yolo_dataset/images/train/{file_id}.jpg'")
        
        # Convert annotations to YOLO format
        df = pd.read_csv(csv_path)
        img_width = df['width'].iloc[0]
        img_height = df['height'].iloc[0]
        
        with open(f"yolo_dataset/labels/train/{file_id}.txt", 'w') as f:
            for _, row in df.iterrows():
                class_name = row['class']
                if class_name not in class_to_idx:
                    continue
                    
                class_id = class_to_idx[class_name]
                x_min, y_min, x_max, y_max = row['xmin'], row['ymin'], row['xmax'], row['ymax']
                
                # Convert to YOLO format (center_x, center_y, width, height) normalized
                x_center = ((x_min + x_max) / 2) / img_width
                y_center = ((y_min + y_max) / 2) / img_height
                width = (x_max - x_min) / img_width
                height = (y_max - y_min) / img_height
                
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
    
    # Process validation files
    for file_id in tqdm(val_files):
        img_path = os.path.join(DATASET_PATH, f"{file_id}.jpg")
        csv_path = os.path.join(DATASET_PATH, f"{file_id}.csv")
        
        if not os.path.exists(img_path) or not os.path.exists(csv_path):
            continue
        
        # Copy image to val folder
        os.system(f"cp '{img_path}' 'yolo_dataset/images/val/{file_id}.jpg'")
        
        # Convert annotations to YOLO format
        df = pd.read_csv(csv_path)
        img_width = df['width'].iloc[0]
        img_height = df['height'].iloc[0]
        
        with open(f"yolo_dataset/labels/val/{file_id}.txt", 'w') as f:
            for _, row in df.iterrows():
                class_name = row['class']
                if class_name not in class_to_idx:
                    continue
                    
                class_id = class_to_idx[class_name]
                x_min, y_min, x_max, y_max = row['xmin'], row['ymin'], row['xmax'], row['ymax']
                
                # Convert to YOLO format (center_x, center_y, width, height) normalized
                x_center = ((x_min + x_max) / 2) / img_width
                y_center = ((y_min + y_max) / 2) / img_height
                width = (x_max - x_min) / img_width
                height = (y_max - y_min) / img_height
                
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
    
    # Create dataset YAML file
    with open('yolo_dataset/dataset.yaml', 'w') as f:
        f.write(f"train: {os.path.abspath('yolo_dataset/images/train')}\n")
        f.write(f"val: {os.path.abspath('yolo_dataset/images/val')}\n")
        f.write(f"nc: {len(class_to_idx)}\n")
        f.write(f"names: {list(class_to_idx.keys())}\n")

# Run preprocessing
print("Creating class mapping...")
class_list, class_to_idx = create_class_mapping()
print(f"Found {len(class_list)} unique classes")

print("\nConverting annotations to YOLO format...")
convert_to_yolo_format(class_to_idx)

## 4. YOLOv12 Model Implementation

In [ ]:
# YOLOv12 Architecture Components

class ConvBNSiLU(nn.Module):
    """Convolution + BatchNorm + SiLU activation"""
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=None, groups=1, bias=False):
        super().__init__()
        if padding is None:
            padding = kernel_size // 2
        self.conv = nn.Conv2d(
            in_channels, out_channels, kernel_size, stride, padding, groups=groups, bias=bias
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.SiLU(inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

class ELAN(nn.Module):
    """Enhanced Layer Aggregation Network for YOLOv12"""
    def __init__(self, in_channels, out_channels, expansion=0.5):
        super().__init__()
        hidden_channels = int(out_channels * expansion)
        self.cv1 = ConvBNSiLU(in_channels, hidden_channels, 1, 1)
        self.cv2 = ConvBNSiLU(hidden_channels, hidden_channels, 3, 1)
        self.cv3 = ConvBNSiLU(hidden_channels, hidden_channels, 3, 1)
        self.cv4 = ConvBNSiLU(hidden_channels, hidden_channels, 3, 1)
        self.cv5 = ConvBNSiLU(hidden_channels, hidden_channels, 3, 1)
        self.cv6 = ConvBNSiLU(hidden_channels * 4, out_channels, 1, 1)
        
    def forward(self, x):
        x = self.cv1(x)
        x2 = self.cv2(x)
        x3 = self.cv3(x2)
        x4 = self.cv4(x3)
        x5 = self.cv5(x4)
        out = self.cv6(torch.cat([x2, x3, x4, x5], dim=1))
        return out

class SPPF(nn.Module):
    """Spatial Pyramid Pooling - Fast"""
    def __init__(self, in_channels, out_channels, kernel_size=5):
        super().__init__()
        hidden_channels = in_channels // 2
        self.cv1 = ConvBNSiLU(in_channels, hidden_channels, 1, 1)
        self.cv2 = ConvBNSiLU(hidden_channels * 4, out_channels, 1, 1)
        self.maxpool = nn.MaxPool2d(kernel_size=kernel_size, stride=1, padding=kernel_size // 2)
        
    def forward(self, x):
        x = self.cv1(x)
        y1 = self.maxpool(x)
        y2 = self.maxpool(y1)
        y3 = self.maxpool(y2)
        out = self.cv2(torch.cat([x, y1, y2, y3], dim=1))
        return out

class DownSample(nn.Module):
    """Downsample module"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = ConvBNSiLU(in_channels, out_channels, 3, 2)
        
    def forward(self, x):
        return self.conv(x)

class CARAFE(nn.Module):
    """Content-Aware ReAssembly of FEatures for upsampling"""
    def __init__(self, in_channels, scale_factor=2, up_kernel=5):
        super().__init__()
        self.scale_factor = scale_factor
        self.up_kernel = up_kernel
        
        self.kernel_conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 1),
            nn.BatchNorm2d(in_channels),
            nn.SiLU(inplace=True),
            nn.Conv2d(in_channels, scale_factor * scale_factor * up_kernel * up_kernel, 1)
        )
        
    def forward(self, x):
        b, c, h, w = x.size()
        # Generate kernel weights
        kernel_weights = self.kernel_conv(x)
        kernel_weights = kernel_weights.view(b, -1, self.up_kernel * self.up_kernel, h, w)
        kernel_weights = F.softmax(kernel_weights, dim=2)
        kernel_weights = kernel_weights.view(b, -1, h, w)
        
        # Upsample feature map
        x = F.interpolate(x, scale_factor=self.scale_factor, mode='nearest')
        kernel_weights = F.interpolate(kernel_weights, scale_factor=self.scale_factor, mode='nearest')
        
        # Apply kernel weights
        x = F.unfold(x, kernel_size=self.up_kernel, padding=self.up_kernel//2)
        x = x.view(b, c, self.up_kernel * self.up_kernel, h * self.scale_factor, w * self.scale_factor)
        kernel_weights = kernel_weights.view(b, self.scale_factor * self.scale_factor, self.up_kernel * self.up_kernel, 
                                           h * self.scale_factor, w * self.scale_factor)
        
        x = torch.sum(x.unsqueeze(2) * kernel_weights.unsqueeze(1), dim=3)
        x = x.view(b, c, h * self.scale_factor, w * self.scale_factor)
        
        return x

class YOLOv12Neck(nn.Module):
    """YOLOv12 Neck with ELAN and SPPF"""
    def __init__(self, in_channels_list, out_channels_list):
        super().__init__()
        self.sppf = SPPF(in_channels_list[2], out_channels_list[2])
        
        # Top-down path
        self.carafe1 = CARAFE(out_channels_list[2])
        self.elan_td1 = ELAN(out_channels_list[2] + in_channels_list[1], out_channels_list[1])
        
        self.carafe2 = CARAFE(out_channels_list[1])
        self.elan_td2 = ELAN(out_channels_list[1] + in_channels_list[0], out_channels_list[0])
        
        # Bottom-up path
        self.down1 = DownSample(out_channels_list[0], out_channels_list[0])
        self.elan_bu1 = ELAN(out_channels_list[0] + out_channels_list[1], out_channels_list[1])
        
        self.down2 = DownSample(out_channels_list[1], out_channels_list[1])
        self.elan_bu2 = ELAN(out_channels_list[1] + out_channels_list[2], out_channels_list[2])
        
    def forward(self, inputs):
        # Backbone features
        x_small, x_medium, x_large = inputs
        
        # SPPF on the largest feature map
        x_large = self.sppf(x_large)
        
        # Top-down path
        p_large = x_large
        p_large_up = self.carafe1(p_large)
        p_medium = self.elan_td1(torch.cat([p_large_up, x_medium], dim=1))
        
        p_medium_up = self.carafe2(p_medium)
        p_small = self.elan_td2(torch.cat([p_medium_up, x_small], dim=1))
        
        # Bottom-up path
        p_small_down = self.down1(p_small)
        p_medium = self.elan_bu1(torch.cat([p_small_down, p_medium], dim=1))
        
        p_medium_down = self.down2(p_medium)
        p_large = self.elan_bu2(torch.cat([p_medium_down, p_large], dim=1))
        
        return p_small, p_medium, p_large

class YOLOv12Head(nn.Module):
    """YOLOv12 Detection Head"""
    def __init__(self, in_channels_list, num_classes, num_anchors=3):
        super().__init__()
        self.num_classes = num_classes
        self.num_anchors = num_anchors
        
        # Detection heads for different scales
        self.det_small = nn.Conv2d(in_channels_list[0], num_anchors * (5 + num_classes), 1)
        self.det_medium = nn.Conv2d(in_channels_list[1], num_anchors * (5 + num_classes), 1)
        self.det_large = nn.Conv2d(in_channels_list[2], num_anchors * (5 + num_classes), 1)
        
    def forward(self, inputs):
        p_small, p_medium, p_large = inputs
        
        # Apply detection heads
        out_small = self.det_small(p_small)
        out_medium = self.det_medium(p_medium)
        out_large = self.det_large(p_large)
        
        # Reshape outputs
        batch_size = out_small.size(0)
        
        out_small = out_small.view(batch_size, self.num_anchors, 5 + self.num_classes, out_small.size(2), out_small.size(3))
        out_small = out_small.permute(0, 1, 3, 4, 2).contiguous()
        
        out_medium = out_medium.view(batch_size, self.num_anchors, 5 + self.num_classes, out_medium.size(2), out_medium.size(3))
        out_medium = out_medium.permute(0, 1, 3, 4, 2).contiguous()
        
        out_large = out_large.view(batch_size, self.num_anchors, 5 + self.num_classes, out_large.size(2), out_large.size(3))
        out_large = out_large.permute(0, 1, 3, 4, 2).contiguous()
        
        return out_small, out_medium, out_large

class YOLOv12Backbone(nn.Module):
    """YOLOv12 Backbone with CSPDarknet"""
    def __init__(self, in_channels=3):
        super().__init__()
        # Initial convolution
        self.conv1 = ConvBNSiLU(in_channels, 32, 3, 1)
        
        # Downsample and ELAN blocks
        self.down1 = DownSample(32, 64)
        self.elan1 = ELAN(64, 64)
        
        self.down2 = DownSample(64, 128)
        self.elan2 = ELAN(128, 128)
        
        self.down3 = DownSample(128, 256)
        self.elan3 = ELAN(256, 256)
        
        self.down4 = DownSample(256, 512)
        self.elan4 = ELAN(512, 512)
        
        self.down5 = DownSample(512, 1024)
        self.elan5 = ELAN(1024, 1024)
        
    def forward(self, x):
        # Initial convolution
        x = self.conv1(x)
        
        # Downsample and ELAN blocks
        x = self.down1(x)
        x = self.elan1(x)
        
        x = self.down2(x)
        x = self.elan2(x)
        out_small = x  # Small scale feature map
        
        x = self.down3(x)
        x = self.elan3(x)
        out_medium = x  # Medium scale feature map
        
        x = self.down4(x)
        x = self.elan4(x)
        
        x = self.down5(x)
        x = self.elan5(x)
        out_large = x  # Large scale feature map
        
        return out_small, out_medium, out_large

class YOLOv12(nn.Module):
    """Complete YOLOv12 model"""
    def __init__(self, num_classes, in_channels=3):
        super().__init__()
        self.backbone = YOLOv12Backbone(in_channels)
        self.neck = YOLOv12Neck([128, 256, 1024], [128, 256, 512])
        self.head = YOLOv12Head([128, 256, 512], num_classes)
        
    def forward(self, x):
        # Get features from backbone
        backbone_features = self.backbone(x)
        
        # Process features through neck
        neck_features = self.neck(backbone_features)
        
        # Get detection outputs from head
        outputs = self.head(neck_features)
        
        return outputs

## 5. Training YOLOv12 Model

In [ ]:
# We'll use Ultralytics YOLO as a base and customize it for YOLOv12
# First, let's create a custom YOLOv12 configuration file

with open('yolov12.yaml', 'w') as f:
    f.write("""
# YOLOv12 configuration for Military Aircraft Detection

# Parameters
nc: 81  # number of classes (will be updated dynamically)
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# Anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv12 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1, 1]],  # 0-P1/2
   [-1, 1, Conv, [64, 3, 2, 1]],  # 1-P2/4
   [-1, 1, ELAN, [64]],
   [-1, 1, Conv, [128, 3, 2, 1]],  # 3-P3/8
   [-1, 1, ELAN, [128]],
   [-1, 1, Conv, [256, 3, 2, 1]],  # 5-P4/16
   [-1, 1, ELAN, [256]],
   [-1, 1, Conv, [512, 3, 2, 1]],  # 7-P5/32
   [-1, 1, ELAN, [512]],
   [-1, 1, Conv, [1024, 3, 2, 1]],  # 9-P6/64
   [-1, 1, ELAN, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 11
  ]

# YOLOv12 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 8], 1, Concat, [1]],  # cat backbone P5
   [-1, 1, ELAN, [512]],  # 15
   
   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 1, ELAN, [256]],  # 19
   
   [-1, 1, Conv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 1, ELAN, [128]],  # 23 (P3/8-small)
   
   [-1, 1, Conv, [128, 3, 2]],
   [[-1, 19], 1, Concat, [1]],  # cat head P4
   [-1, 1, ELAN, [256]],  # 26 (P4/16-medium)
   
   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 15], 1, Concat, [1]],  # cat head P5
   [-1, 1, ELAN, [512]],  # 29 (P5/32-large)
   
   [[23, 26, 29], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]
""")

# Update the number of classes in the YAML file
import yaml
with open('yolov12.yaml', 'r') as f:
    yolo_config = yaml.safe_load(f)

yolo_config['nc'] = len(class_to_idx)

with open('yolov12.yaml', 'w') as f:
    yaml.dump(yolo_config, f, default_flow_style=False)

In [ ]:
# Train YOLOv12 model using Ultralytics YOLO
!yolo task=detect mode=train model=yolov12.yaml data=yolo_dataset/dataset.yaml epochs=50 imgsz=640 batch=16 name=yolov12_military_aircraft

## 6. Model Evaluation

In [ ]:
# Evaluate the trained model
!yolo task=detect mode=val model=runs/detect/yolov12_military_aircraft/weights/best.pt data=yolo_dataset/dataset.yaml

In [ ]:
# Visualize some predictions
def visualize_predictions(model_path, image_paths, conf_threshold=0.25):
    model = YOLO(model_path)
    
    plt.figure(figsize=(20, 20))
    for i, img_path in enumerate(image_paths):
        if i >= 9:  # Limit to 9 images
            break
            
        results = model(img_path, conf=conf_threshold)[0]
        
        # Get the image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Plot in a 3x3 grid
        plt.subplot(3, 3, i+1)
        plt.imshow(img)
        
        # Draw bounding boxes
        for box in results.boxes.xyxy:
            x1, y1, x2, y2 = box.tolist()
            plt.gca().add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                                             fill=False, edgecolor='red', linewidth=2))
        
        # Add class labels and confidence scores
        for box, cls, conf in zip(results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
            x1, y1, x2, y2 = box.tolist()
            class_name = results.names[int(cls.item())]
            confidence = conf.item()
            plt.text(x1, y1-10, f"{class_name}: {confidence:.2f}", color='red', fontsize=12, 
                     bbox=dict(facecolor='white', alpha=0.7))
        
        plt.title(f"Image {i+1}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Get some validation images
val_images = glob.glob('yolo_dataset/images/val/*.jpg')[:9]
visualize_predictions('runs/detect/yolov12_military_aircraft/weights/best.pt', val_images)

## 7. Inference with YOLOv12

In [ ]:
# Function to perform inference on new images
def detect_aircraft(model_path, image_path, conf_threshold=0.25):
    model = YOLO(model_path)
    results = model(image_path, conf=conf_threshold)[0]
    
    # Get the image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Create figure
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    
    # Draw bounding boxes
    for box, cls, conf in zip(results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
        x1, y1, x2, y2 = box.tolist()
        class_name = results.names[int(cls.item())]
        confidence = conf.item()
        
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                                         fill=False, edgecolor='red', linewidth=2))
        plt.text(x1, y1-10, f"{class_name}: {confidence:.2f}", color='red', fontsize=12, 
                 bbox=dict(facecolor='white', alpha=0.7))
    
    plt.title(f"Aircraft Detection: {os.path.basename(image_path)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Print detection results
    print(f"Detected {len(results.boxes)} aircraft:")
    for i, (cls, conf) in enumerate(zip(results.boxes.cls, results.boxes.conf)):
        class_name = results.names[int(cls.item())]
        confidence = conf.item()
        print(f"  {i+1}. {class_name} (Confidence: {confidence:.2f})")

# Try inference on a sample image
sample_image = val_images[0] if val_images else None
if sample_image:
    detect_aircraft('runs/detect/yolov12_military_aircraft/weights/best.pt', sample_image)

## 8. Export Model for Deployment

In [ ]:
# Export the model to different formats
!yolo export model=runs/detect/yolov12_military_aircraft/weights/best.pt format=onnx

## 9. Conclusion and Usage Instructions

This notebook has implemented YOLOv12 for the Military Aircraft Detection Dataset. The implementation includes:

1. Data preprocessing and conversion to YOLO format
2. YOLOv12 model architecture implementation
3. Model training and evaluation
4. Inference examples
5. Model export for deployment

### How to Use the Model

To use the trained model for inference on new images:

```python
from ultralytics import YOLO

# Load the model
model = YOLO('runs/detect/yolov12_military_aircraft/weights/best.pt')

# Perform inference
results = model('path/to/image.jpg')

# Process results
for result in results:
    boxes = result.boxes  # Boxes object for bounding boxes outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
```

### Model Performance

The model's performance metrics are available in the evaluation section. The key metrics include:

- mAP (mean Average Precision)
- Precision
- Recall
- F1-Score

### Further Improvements

To further improve the model performance, consider:

1. Increasing the training epochs
2. Applying more data augmentation techniques
3. Fine-tuning hyperparameters
4. Using transfer learning from a pre-trained model
5. Implementing ensemble methods with multiple models